# Accessing the NHTSA API [crawler]
## Complaints Data 

In [145]:
import requests
from pathlib import Path
import json
import pandas as pd
import urllib.request
import urllib.error
import time
from loguru import logger
from IPython.display import display, clear_output
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Set API prefix because our api headers are same.
api_prefix = "https://api.nhtsa.gov"

# Set API suffix for get year.
api_get_year = "/products/vehicle/modelYears?issueType=c"

# Set output path.
dir_raw = Path("../raw/Complaints-Plus")
dir_raw.mkdir(parents=True, exist_ok=True)

In [261]:
def fetch_complaints(url):
    logger.info("Fetching %s" % url)
    try:
        headers = {
          'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
        }
        response = requests.get(url,headers=headers)
        status = response.status_code
        logger.info(status)
        
        # Stop if the API cannot be accessed properly.
        if status != 200:
            return [-1,None]
        
        # Otherwise return retrieved data.
        raw_data = response.text
        return [1,json.loads(raw_data)]
    except urllib.error.URLError as e:
        e.reason 

# Output each complaint for a json file.
def write_out_raw(complaints_count,year,make,model,data):
    # Check if '/' is already in make or model, replaced by '_or_'.
    if '/' in make:
        make = make.replace("/", "_or_")
    if '/' in model:
        model = model.replace("/", "_or_")
    # File name format [complaint_num]year_make_model.json
    #             e.g. [802]1994_MERCEDES BENZ_E.json
    fname = "[%d]%s_%s_%s.json" % (complaints_count,year,make,model)
    out_path = dir_raw / fname
    logger.info("Writing data to %s" % out_path)
    fout = open(out_path, "w")
    json.dump(data, fout, indent=4, sort_keys=True)
    fout.close()
    
# Check API status.
def check_status(data):
    # Number of complaint in this time.
    complaint_num = data["count"]
    # Are results returned successfully?
    if data["message"] != "Results returned successfully":
        logger.warning("Fail to get from API, %s" % (data["status_message"]))
        return [False,-1]
    elif complaint_num <= 0:
        logger.warning("No info from current API address.")
        return [False,0]
    else:
        return [True,complaint_num]

# Output for "years", "makes", "models" which are dictionaries we get from first 3 steps.  
def write_out(name,data):
    fname = "%s.json" % (name)
    out_dir = Path("../raw/Complaints_buffer")
    out_path = out_dir / fname
    logger.info("Writing data to %s" % out_path)
    fout = open(out_path, "w")
    json.dump(data, fout, indent=4, sort_keys=True)
    fout.close()

## Step-1: Get all Model Years 

In [6]:
# Construct API url.
api = api_prefix + api_get_year
# Fetch data.
status,data = fetch_complaints(api)
# Convert result data into list format.
years = (list(each["modelYear"] for each in data["results"]))
logger.info("Completed the task of getting all model years!")
# Store years data.
write_out("years",years)

2022-04-05 21:11:35.238 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/products/vehicle/modelYears?issueType=c
2022-04-05 21:11:35.587 | INFO     | __main__:fetch_complaints:9 - 200


## Step-2: Get all Makes for the Model Year

In [7]:
makes={}
for year in years:
    clear_output()
    # Construct API url.
    api = api_prefix + "/products/vehicle/makes?modelYear=%d&issueType=c" % (int(year))
    # Fetch data.
    status,data = fetch_complaints(api)
    # Convert result data into list format then store in a dict by year.
    makes[year] = (list(each["make"] for each in data["results"]))
logger.info("Completed the task of getting all makes for the model year!")
# Store makes data.
write_out("complaint_makes",makes)

2022-04-05 21:12:33.344 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/products/vehicle/makes?modelYear=9999&issueType=c
2022-04-05 21:12:34.148 | INFO     | __main__:fetch_complaints:9 - 200


## Step-3: Get all Models for the Make and Model Year 
Due to the website's restrictions on crawlers, we have to change IP frequently to bypass its restrictions. The following function will automatically stop when it received an access error (Typically a 403 error) and return the data we got so far. And it will give us a hint for next start.

In [53]:
def Get_model(years,skip):
    models={}
    start_year = years[0]
    for year in years:
        for make in makes[year]:
            # If meet some special case we need skip it.
            if make != skip:
                clear_output()
                api = api_prefix + "/products/vehicle/models?modelYear=%d&make=%s&issueType=c" % (int(year),make)
                status,data = fetch_complaints(api)
            # Status equal -1 means we are failing to use this API at this moment.
            if status == -1:
                logger.warning("Start year:%s, Current Year:%s(index:%d) Current Make:%s(index:%d)" % (start_year,year,years.index(year),make,makes[year].index(make)))
                return models
            # Otherwise, convert result data into list format then store in a dict by make.
            else:
                models[make] = (list(each["model"] for each in data["results"]))
    end_year = year
    logger.info("Finished (%s-%s)" % (start_year,end_year))
    return models

#### We cannot access url related to **"EMERGENCY ONE"**, therefore we skip it.

In [54]:
models = Get_model(years,"EMERGENCY ONE")

2022-04-05 23:43:22.630 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/products/vehicle/models?modelYear=2011&make=NU WA&issueType=c
2022-04-05 23:43:22.809 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-05 23:43:22.810 | WARNING  | __main__:Get_model:13 - Start year:1949, Current Year:2011(index:56) Current Make:NU WA(index:102)


In [57]:
start = years.index("2011")
model_2 = Get_model(years[start:],"EMERGENCY ONE")

2022-04-05 23:51:34.639 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/products/vehicle/models?modelYear=9999&make=ZAP&issueType=c
2022-04-05 23:51:34.837 | INFO     | __main__:fetch_complaints:9 - 200
2022-04-05 23:51:34.838 | INFO     | __main__:Get_model:18 - Finished (2011-9999)


In [63]:
len(models),len(model_2)

(709, 517)

#### Define a function for merge two dictionary with out replacement & duplication.

In [60]:
def merge_2_dict(dict_1,dict_2):
    for key, value in dict_2.items():
        if key in dict_1:
            for v in value:
                if v not in dict_1[key]:
                    dict_1[key].append(v)
        else:
            dict_1[key] = value
    return dict_1

In [70]:
models = merge_2_dict(models,model_2)

Delete pair if the value of this key is empty.

In [71]:
len(models)

934

In [72]:
clean_models = models.copy()
for key in list(clean_models.keys()):
    if not clean_models.get(key):
        del clean_models[key]
len(clean_models)

860

#### Delect special 403 case from **makes** and **models**.

In [228]:
#Retrieve back from file

#f = open('../raw/Complaints-Plus/makes.json')
#data_m = json.load(f)
#f.close()
#makes = data_m.copy()

In [229]:
makes_clean_403 = makes.copy()
for k,v in makes_clean_403.items():
    if "EMERGENCY ONE" in v:
        id = v.index("EMERGENCY ONE")
        del v[id]
logger.info( "%d -> %d" %(len(makes),len(makes_clean_403)))

2022-04-06 15:14:27.720 | INFO     | __main__:<module>:6 - 69 -> 69


In [230]:
models_clean_403 = models.copy()
del models_clean_403["EMERGENCY ONE"]
logger.info( "%d -> %d" %(len(models),len(models_clean_403)))

2022-04-06 15:14:32.666 | INFO     | __main__:<module>:3 - 934 -> 933


In [77]:
write_out("models",models)

2022-04-06 00:04:51.999 | INFO     | __main__:write_out:44 - Writing data to ../raw/Complaints-Plus/models.json
2022-04-06 00:04:52.006 | INFO     | __main__:write_out:44 - Writing data to ../raw/Complaints-Plus/makes.json
2022-04-06 00:04:52.013 | INFO     | __main__:write_out:44 - Writing data to ../raw/Complaints-Plus/years.json


## Step-4: Get all complaints for the selected Model Year, Make, Model

In [146]:
def Get_complaints(complaints_count,years,makes,models,continue_):
    complaints_count = complaints_count
    start_year = years[0]
    for year in years:
        # "continue_" means we've suffered an interruption, 
        # and we're going back to that position.
        if continue_ : 
            #Get user input of "make" which want to continue.
            make_input = input()
            
            #Find correspond index position in the "makes" dict.
            position_make = makes[year].index(make_input)
            
            #Get user input of "model" which want to continue.
            model_input = input()
            
            #Find correspond index position in the "models" dict.
            position_model = models[make_input].index(model_input)
            
            # Find a list of rest of makes by current year. 
            make_position = makes[year].index(make_input)
            
            # Find a list of rest of models by current year & make. 
            model_position = models[make_input].index(model_input)

            for make in (makes[year])[make_position:]:
                for model in (models[make_input])[model_position:]:
                    
                    clear_output()
                    api = api_prefix + "/complaints/complaintsByVehicle?make=%s&model=%s&modelYear=%d" % (make,model,int(year))
                    status, data = fetch_complaints(api)
                    
                    # Status equal -1 means we are failing to use this API at this moment.
                    if status == -1:
                        logger.warning("Stopped! (%s_%s_%s_%s)" % (complaints_count,year,make,model))                    
                        alarm_clock()
                        return -1
                    
                    # Skip if data is empty
                    if data["results"] != []:
                        #Write out
                        write_out_raw(complaints_count,year,make,model,data["results"])
                        complaints_count += 1
                    else:
                        logger.info("Empty data")
            continue_ = False
        else:
            for make in makes[year]:
                for model in models[make]:
                    clear_output()
                    api = api_prefix + "/complaints/complaintsByVehicle?make=%s&model=%s&modelYear=%d" % (make,model,int(year))
                    # Fetch data.
                    status, data = fetch_complaints(api)

                    # Status equal -1 means we are failing to use this API at this moment.
                    if status == -1:
                        logger.warning("Stopped! (%s_%s_%s_%s)" % (complaints_count,year,make,model)) 
                        alarm_clock()
                        return -1
                    
                    # Skip if data is empty
                    if data["results"] != []:
                        #Write out
                        write_out_raw(complaints_count,year,make,model,data["results"])
                        complaints_count += 1
                    else:
                        logger.info("Empty data")
    end_year = year
    logger.info("Finished (%s_%s_%s_%s) from %s to %s." % (complaints_count,year,make,model,start_year,end_year))
    
# Set an alert to remind me to reset when our IP is blocked.
def alarm_clock():
    file = 'alarm_music.mp3'
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    time.sleep(10)
    pygame.mixer.music.stop()

In [97]:
Get_complaints(0,years,makes,models,False)

2022-04-06 00:27:28.693 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=CADILLAC&model=STS&modelYear=1972
2022-04-06 00:27:28.898 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 00:27:28.900 | WARNING  | __main__:Get_complaints:36 - Stopped! (27_1972_CADILLAC_STS)


-1

In [135]:
start = years.index("1972")
Get_complaints(20,years[start:],makes,models,True)

2022-04-06 12:27:26.238 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=BMW&model=550I&modelYear=1978
2022-04-06 12:27:26.286 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 12:27:26.288 | WARNING  | __main__:Get_complaints:53 - Stopped! (49_1978_BMW_550I)


-1

In [141]:
start = years.index("1978")
Get_complaints(49,years[start:],makes,models,True)

2022-04-06 13:32:34.714 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=MERCEDES BENZ&model=ML350&modelYear=1979
2022-04-06 13:32:34.895 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 13:32:34.896 | WARNING  | __main__:Get_complaints:56 - Stopped! (63_1979_MERCEDES BENZ_ML350)


-1

In [143]:
start = years.index("1979")
Get_complaints(63,years[start:],makes,models,True)

2022-04-06 14:06:41.186 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=FORD&model=FLEX&modelYear=1986
2022-04-06 14:06:41.381 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 14:06:41.382 | WARNING  | __main__:Get_complaints:56 - Stopped! (236_1986_FORD_FLEX)


-1

In [231]:
start = years.index("1986")
Get_complaints(236,years[start:],makes,models,True)

2022-04-06 16:07:22.501 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=CONQUEST&model=TRAILER&modelYear=1995
2022-04-06 16:07:22.670 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 16:07:22.671 | WARNING  | __main__:Get_complaints:57 - Stopped! (863_1995_CONQUEST_TRAILER)


-1

In [234]:
start = years.index("1995")
Get_complaints(863,years[start:],makes,models,True)

2022-04-06 17:07:06.153 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=SUZUKI&model=GRAND VITARA&modelYear=1997
2022-04-06 17:07:06.334 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 17:07:06.335 | WARNING  | __main__:Get_complaints:57 - Stopped! (1129_1997_SUZUKI_GRAND VITARA)


-1

In [235]:
start = years.index("1997")
Get_complaints(1129,years[start:],makes,models,True)

2022-04-06 18:15:40.172 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=SKYLINE&model=RAMPAGE&modelYear=2004
2022-04-06 18:15:40.343 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 18:15:40.344 | WARNING  | __main__:Get_complaints:57 - Stopped! (2566_2004_SKYLINE_RAMPAGE)


-1

In [236]:
start = years.index("2004")
Get_complaints(2566,years[start:],makes,models,True)

2022-04-06 18:47:32.905 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=JAGUAR&model=XF&modelYear=2007
2022-04-06 18:47:33.091 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 18:47:33.092 | WARNING  | __main__:Get_complaints:57 - Stopped! (3341_2007_JAGUAR_XF)


-1

In [237]:
start = years.index("2007")
Get_complaints(3341,years[start:],makes,models,True)

2022-04-06 19:58:23.882 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=CHEVROLET&model=SILVERADO 2500&modelYear=2009
2022-04-06 19:58:24.054 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 19:58:24.056 | WARNING  | __main__:Get_complaints:57 - Stopped! (3781_2009_CHEVROLET_SILVERADO 2500)


-1

In [238]:
start = years.index("2009")
Get_complaints(3781,years[start:],makes,models,True)

2022-04-06 20:12:33.888 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=SUZUKI&model=SILVERADO 3500&modelYear=2009
2022-04-06 20:12:34.113 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 20:12:34.114 | WARNING  | __main__:Get_complaints:36 - Stopped! (3787_2009_SUZUKI_SILVERADO 3500)


-1

In [242]:
start = years.index("2009")
Get_complaints(3787,years[start:],makes,models,True)

2022-04-06 20:32:20.144 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=BMW&model=X5&modelYear=2011
2022-04-06 20:32:20.322 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 20:32:20.323 | WARNING  | __main__:Get_complaints:57 - Stopped! (4291_2011_BMW_X5)


-1

In [243]:
start = years.index("2011")
Get_complaints(4291,years[start:],makes,models,True)

2022-04-06 20:47:25.966 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=ASTON MARTIN&model=V8 VANTAGE S&modelYear=2012
2022-04-06 20:47:26.143 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 20:47:26.143 | WARNING  | __main__:Get_complaints:57 - Stopped! (4303_2012_ASTON MARTIN_V8 VANTAGE S)


-1

In [245]:
start = years.index("2012")
Get_complaints(4303,years[start:],makes,models,True)

2022-04-06 21:12:31.442 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=MERCEDES BENZ&model=E&modelYear=2013
2022-04-06 21:12:31.617 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 21:12:31.618 | WARNING  | __main__:Get_complaints:57 - Stopped! (4572_2013_MERCEDES BENZ_E)


-1

In [246]:
start = years.index("2013")
Get_complaints(4572,years[start:],makes,models,True)

2022-04-06 21:22:34.743 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=UTILITY&model=GL350&modelYear=2013
2022-04-06 21:22:34.999 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 21:22:35.000 | WARNING  | __main__:Get_complaints:36 - Stopped! (4572_2013_UTILITY_GL350)


-1

In [251]:
start = years.index("2013")
Get_complaints(4572,years[start:],makes,models,True)

2022-04-06 21:37:27.173 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=FORD&model=FLEX&modelYear=2015
2022-04-06 21:37:27.372 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 21:37:27.373 | WARNING  | __main__:Get_complaints:57 - Stopped! (5022_2015_FORD_FLEX)


-1

In [253]:
start = years.index("2015")
Get_complaints(5022,years[start:],makes,models,True)

2022-04-06 21:47:30.216 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=YAMAHA&model=FUSION HYBRID&modelYear=2015
2022-04-06 21:47:30.391 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 21:47:30.392 | WARNING  | __main__:Get_complaints:36 - Stopped! (5029_2015_YAMAHA_FUSION HYBRID)


-1

In [256]:
start = years.index("2015")
Get_complaints(5029,years[start:],makes,models,True)

2022-04-06 21:57:24.063 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=VOLVO&model=780&modelYear=2016
2022-04-06 21:57:24.228 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 21:57:24.229 | WARNING  | __main__:Get_complaints:57 - Stopped! (5332_2016_VOLVO_780)


-1

In [257]:
start = years.index("2016")
Get_complaints(5332,years[start:],makes,models,True)

2022-04-06 22:38:23.834 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=LANCE&model=2281&modelYear=2021
2022-04-06 22:38:24.021 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 22:38:24.022 | WARNING  | __main__:Get_complaints:57 - Stopped! (6445_2021_LANCE_2281)


-1

In [259]:
start = years.index("2021")
Get_complaints(6445,years[start:],makes,models,True)

2022-04-06 22:47:23.454 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=TRIUMPH&model=THUNDERBIRD ABS A1&modelYear=9999
2022-04-06 22:47:23.634 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 22:47:23.635 | WARNING  | __main__:Get_complaints:57 - Stopped! (6533_9999_TRIUMPH_THUNDERBIRD ABS A1)


-1

In [260]:
start = years.index("9999")
Get_complaints(6533,years[start:],makes,models,True)

2022-04-06 22:48:38.660 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=ZAP&model=TIGER 1050 CCM ABS&modelYear=9999
2022-04-06 22:48:38.900 | INFO     | __main__:fetch_complaints:9 - 200
2022-04-06 22:48:38.901 | INFO     | __main__:Get_complaints:45 - Empty data
2022-04-06 22:48:38.902 | INFO     | __main__:Get_complaints:68 - Finished (6533_9999_ZAP_TIGER 1050 CCM ABS) from 9999 to 9999.
